<a href="https://colab.research.google.com/github/Tejaswini1309/Satellite-Imagery-Based-Property-Valuation/blob/main/data_fetcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

PROJECT_PATH = "/content/drive/MyDrive/project_cdc"
os.chdir(PROJECT_PATH)

print("Current directory:", os.getcwd())

In [ ]:
import requests
import pandas as pd
from PIL import Image
import io
import matplotlib.pyplot as plt
from io import BytesIO



In [ ]:
df = pd.read_csv("train1.csv")
df.head()


In [ ]:
row = df.iloc[20]

lat = row["lat"]
lon = row["long"]
house_id = row["id"]


In [ ]:
def get_bbox(lat, lon, delta=0.003):
    """
    delta ~ 0.01 ≈ ~1 km
    total image ≈ 2 km x 2 km
    """
    min_lon = lon - delta
    min_lat = lat - delta
    max_lon = lon + delta
    max_lat = lat + delta
    return f"{min_lon},{min_lat},{max_lon},{max_lat}"


In [ ]:
def fetch_esri_image(lat, lon, save_path, size=224):
    url = "https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export"

    bbox = get_bbox(lat, lon)

    params = {
        "bbox": bbox,
        "bboxSR": 4326,
        "imageSR": 4326,
        "size": f"{size},{size}",
        "format": "png",
        "f": "image"
    }

    response = requests.get(url, params=params)
    response.raise_for_status()

    img = Image.open(BytesIO(response.content)).convert("RGB")
    img.save(save_path)


In [ ]:
BASE_DIR = "images/train"
os.makedirs(BASE_DIR, exist_ok=True)

In [ ]:
x=0
for i in range(df.shape[0]):
    row = df.iloc[i]

    house_id = str(row["id"])
    lat = row["lat"]
    lon = row["long"]

    save_path = os.path.join(BASE_DIR, f"{house_id}.png")

    if os.path.exists(save_path):
        x+=i
        if i % 100 == 0:
            print(f"Processed {i}/{len(df)}")
        continue

    try:
        fetch_esri_image(
            lat=lat,
            lon=lon,
            save_path=save_path,
            size=224
        )



    except Exception as e:
        print(f" Failed {house_id}: {e}")
print(x)

In [ ]:
ddf = pd.read_csv("test2.csv")
ddf.head()

In [ ]:
BASE_DIR = "images/test"
os.makedirs(BASE_DIR, exist_ok=True)

In [ ]:
for i in range(ddf.shape[0]):
    row = ddf.iloc[i]

    house_id = str(row["id"])
    lat = row["lat"]
    lon = row["long"]

    save_path = os.path.join(BASE_DIR, f"{house_id}.png")

    if os.path.exists(save_path):
        continue

    try:
        fetch_esri_image(
            lat=lat,
            lon=lon,
            save_path=save_path,
            size=224
        )

        if i % 100 == 0:
            print(f"Processed {i}/{len(ddf)}")

    except Exception as e:
        print(f"Failed {house_id}: {e}")